In [ ]:
from RandomTesting.__RandomTesting__ import Random_Testing, calculate_extract_loss
from tqdm import tqdm
from sklearn import neighbors
from pykrige.ok import OrdinaryKriging
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
import pandas as pd
import gc


warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = ""
tf.config.optimizer.set_jit(True)

# Kriging Random Testing 

In [ ]:
extract_station_nb = 1 #1,2,3,4

EPA73_test_pm25 = np.load('../EPA73_Testing_348X204_PM25.npy', mmap_mode='r')
station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
ex5_lst = np.load('RandomTesting/extract5_list.npy')
ex10_lst = np.load('RandomTesting/extract10_list.npy')
ex15_lst = np.load('RandomTesting/extract15_list.npy')
ex20_lst = np.load('RandomTesting/extract20_list.npy')

grid_lon = np.array(np.arange(204),dtype='float32')
grid_lat = np.array(np.arange(348),dtype='float32') 

total_mse, total_mae, total_mape = ([] for _ in range(3))

for qq in range(30):
    matrix20,lats_ex20,lons_ex20=Random_Testing(extract_station_nb, qq, station_coordinate, EPA73_test_pm25, 
                                                ex5_lst, ex10_lst, ex15_lst, ex20_lst)
    total_result = []
    for i in tqdm(range(matrix20.shape[0])):
        value_lst = [] 

        lat_all = (np.where(matrix20[i,:,:,0] != 0)[0])
        lon_all = (np.where(matrix20[i,:,:,0] != 0)[1])
        
        for lat, lon in zip(lat_all, lon_all): value_lst.append(matrix20[i,lat,lon,0])

        matrix_lon_array = np.array(lon_all,dtype='float32')
        matrix_lat_array = np.array(lat_all,dtype='float32')       
        train_x = np.array(np.vstack((matrix_lon_array,matrix_lat_array)).T,dtype='float32')    
        train_y = np.array(value_lst,dtype='float32')

        OK = OrdinaryKriging(train_x[:,0], train_x[:,1], train_y, variogram_model='linear', weight = True,
                             exact_values = False, verbose=False)

        values, ss1 = OK.execute('grid', grid_lon, grid_lat)
        total_result.append(values)
        
    mse,mae,mape = calculate_extract_loss(np.array(total_result),EPA73_test_pm25[:,:,:,0],lats_ex20,lons_ex20)
    total_mse.append(mse)
    total_mae.append(mae)
    total_mape.append(mape)

print(np.array(total_mse).max())
print(np.array(total_mse).min())
print(np.array(total_mse).mean())
print(np.array(total_mse).var())
print(np.array(total_mse).std())